In [1]:
import librosa
import IPython.display as ipd
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import sklearn as sk
import pandas as pd
from sklearn import preprocessing
import wav_to_mel_spect
import os

In [2]:
PATHSAMPLES = '../data/training_list.txt'
NSAMPLES = 3000
SOUNDS =np.array(['yes','no','right','five','nine'])
path_out = '/home/josearangos/Documentos/Projects/Voice_Recognition/data/mel-spectrograms'

In [3]:
def get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS):
    ROOTPATH= 'data/'
    data = pd.read_csv(PATHSAMPLES, sep=" ", header=None)
    data = data.values #Convertir en numpy array
    labels = []
    speakers  = []
    paths = []
    for path in data:
        pathSplit =path[0].split('/')
        speakerName = pathSplit[1].split('_')[0]
        word = pathSplit[0]
        isWord = np.where(SOUNDS == word)
        if len(isWord[0]) != 0:
            label = SOUNDS[isWord[0][0]]
            labels.append(label)
            speakers.append(speakerName)
            paths.append(ROOTPATH+path[0])
    NSAMPLES = NSAMPLES
    first = labels.index(SOUNDS[0])
    last = first +NSAMPLES
    matrixMajor = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
    for i in range(1,np.size(SOUNDS,0)):
        first = labels.index(SOUNDS[i])
        last = first +NSAMPLES
        clasei = np.column_stack([paths[first:last], labels[first:last], speakers[first:last]])    
        matrixMajor = np.append(matrixMajor,clasei,axis=0)
    return matrixMajor

In [4]:
path_word_speakerId = get_Path_Word_SpeakerId(NSAMPLES, PATHSAMPLES, SOUNDS)

In [5]:
path_word_speakerId.shape

(15000, 3)

In [6]:
path_word_speakerId

array([['data/yes/004ae714_nohash_0.wav', 'yes', '004ae714'],
       ['data/yes/004ae714_nohash_1.wav', 'yes', '004ae714'],
       ['data/yes/00970ce1_nohash_0.wav', 'yes', '00970ce1'],
       ...,
       ['data/nine/f2b8fc18_nohash_0.wav', 'nine', 'f2b8fc18'],
       ['data/nine/f2e9b610_nohash_0.wav', 'nine', 'f2e9b610'],
       ['data/nine/f2e9b610_nohash_1.wav', 'nine', 'f2e9b610']],
      dtype='<U33')

In [7]:
def build_features(path_word_speakerId,SOUNDS):
    
    for s in SOUNDS:
        path_directory = path_out+'/'+s
        if not os.path.exists(path_directory):
            os.mkdir(path_directory)    
            print("Directory " , path_directory ,  " Created ")
        else:    
            print("Directory " , path_directory ,  " already exists")
            
    for p in path_word_speakerId:
        if p[1]!='yes' or p[1]!='no' :
            path = '../'+p[0]       
            paht_direct = path_out+'/'+p[1]

            wav_to_mel_spect.wav2melSpectIm(path, paht_direct)

In [ ]:
build_features(path_word_speakerId,SOUNDS)

Directory  /home/josearangos/Documentos/Projects/Voice_Recognition/data/mel-spectrograms/yes  already exists
Directory  /home/josearangos/Documentos/Projects/Voice_Recognition/data/mel-spectrograms/no  already exists
Directory  /home/josearangos/Documentos/Projects/Voice_Recognition/data/mel-spectrograms/right  already exists
Directory  /home/josearangos/Documentos/Projects/Voice_Recognition/data/mel-spectrograms/five  already exists
Directory  /home/josearangos/Documentos/Projects/Voice_Recognition/data/mel-spectrograms/nine  already exists
